# 这章讲的是SLAM的回环检测

词袋模型就是将一些特征点，组合起来有一定的特征，再通过这些特征进行匹配和验证

相似度的计算原理：是通过欧式距离等方式进行处理的

   gtsam::NonlinearFactorGraph::shared_ptr graph ( new gtsam::NonlinearFactorGraph );  // gtsam的因子图
    gtsam::Values::shared_ptr initial ( new gtsam::Values ); // 初始值
    // 从g2o文件中读取节点和边的信息
    int cntVertex=0, cntEdge = 0;
    cout<<"reading from g2o file"<<endl;
    
    while ( !fin.eof() )
    {
        string tag;
        fin>>tag;
        if ( tag == "VERTEX_SE3:QUAT" )
        {
            // 顶点
            gtsam::Key id;
            fin>>id;
            double data[7];
            for ( int i=0; i<7; i++ ) fin>>data[i];
            // 转换至gtsam的Pose3
            gtsam::Rot3 R = gtsam::Rot3::Quaternion ( data[6], data[3], data[4], data[5] );
            gtsam::Point3 t ( data[0], data[1], data[2] );
            initial->insert ( id, gtsam::Pose3 ( R,t ) );       // 添加初始值
            cntVertex++;
        }
        else if ( tag == "EDGE_SE3:QUAT" )
        {
            // 边，对应到因子图中的因子
            gtsam::Matrix m = gtsam::I_6x6;     // 信息矩阵
            gtsam::Key id1, id2;
            fin>>id1>>id2;
            double data[7];
            for ( int i=0; i<7; i++ ) fin>>data[i];
            gtsam::Rot3 R = gtsam::Rot3::Quaternion ( data[6], data[3], data[4], data[5] );
            gtsam::Point3 t ( data[0], data[1], data[2] );
            for ( int i=0; i<6; i++ )
                for ( int j=i; j<6; j++ )
                {
                    double mij;
                    fin>>mij;
                    m ( i,j ) = mij;
                    m ( j,i ) = mij;
                }
                
            // g2o的信息矩阵定义方式与gtsam不同，这里对它进行修改
            gtsam::Matrix mgtsam = gtsam::I_6x6;
            mgtsam.block<3,3> ( 0,0 ) = m.block<3,3> ( 3,3 ); // cov rotation
            mgtsam.block<3,3> ( 3,3 ) = m.block<3,3> ( 0,0 ); // cov translation
            mgtsam.block<3,3> ( 0,3 ) = m.block<3,3> ( 0,3 ); // off diagonal
            mgtsam.block<3,3> ( 3,0 ) = m.block<3,3> ( 3,0 ); // off diagonal
            
            gtsam::SharedNoiseModel model = gtsam::noiseModel::Gaussian::Information ( mgtsam );        // 高斯噪声模型
            gtsam::NonlinearFactor::shared_ptr factor ( 
                new gtsam::BetweenFactor<gtsam::Pose3> ( id1, id2, gtsam::Pose3 ( R,t ), model ) // 添加一个因子
            );
            graph->push_back ( factor );
            cntEdge++;
        }
        if ( !fin.good() )
            break;
    }
    
    cout<<"read total "<<cntVertex<<" vertices, "<<cntEdge<<" edges."<<endl;
    // 固定第一个顶点，在gtsam中相当于添加一个先验因子 
    gtsam::NonlinearFactorGraph graphWithPrior = *graph;
    gtsam::noiseModel::Diagonal::shared_ptr priorModel = 
        gtsam::noiseModel::Diagonal::Variances (
            ( gtsam::Vector ( 6 ) <<1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6 ).finished() 
        );
    gtsam::Key firstKey = 0;
    for ( const gtsam::Values::ConstKeyValuePair& key_value: *initial )
    {
        cout<<"Adding prior to g2o file "<<endl;
        graphWithPrior.add ( gtsam::PriorFactor<gtsam::Pose3> ( 
            key_value.key, key_value.value.cast<gtsam::Pose3>(), priorModel ) 
        );
        break;
    }

    // 开始因子图优化，配置优化选项
    cout<<"optimizing the factor graph"<<endl;
    // 我们使用 LM 优化
    gtsam::LevenbergMarquardtParams params_lm;
    params_lm.setVerbosity("ERROR");
    params_lm.setMaxIterations(20);
    params_lm.setLinearSolverType("MULTIFRONTAL_QR");
    gtsam::LevenbergMarquardtOptimizer optimizer_LM( graphWithPrior, *initial, params_lm );
    
    // 你可以尝试下 GN
    // gtsam::GaussNewtonParams params_gn;
    // params_gn.setVerbosity("ERROR");
    // params_gn.setMaxIterations(20);
    // params_gn.setLinearSolverType("MULTIFRONTAL_QR");
    // gtsam::GaussNewtonOptimizer optimizer ( graphWithPrior, *initial, params_gn );
    
    gtsam::Values result = optimizer_LM.optimize();
    cout<<"Optimization complete"<<endl;
    cout<<"initial error: "<<graph->error ( *initial ) <<endl;
    cout<<"final error: "<<graph->error ( result ) <<endl;

    cout<<"done. write to g2o ... "<<endl;
    // 写入 g2o 文件，同样伪装成 g2o 中的顶点和边，以便用 g2o_viewer 查看。
    // 顶点咯
    ofstream fout ( "result_gtsam.g2o" );
    for ( const gtsam::Values::ConstKeyValuePair& key_value: result )
    {
        gtsam::Pose3 pose = key_value.value.cast<gtsam::Pose3>();
        gtsam::Point3 p = pose.translation();
        gtsam::Quaternion q = pose.rotation().toQuaternion();
        fout<<"VERTEX_SE3:QUAT "<<key_value.key<<" "
            <<p.x() <<" "<<p.y() <<" "<<p.z() <<" "
            <<q.x()<<" "<<q.y()<<" "<<q.z()<<" "<<q.w()<<" "<<endl;
    }
    // 边咯 
    for ( gtsam::NonlinearFactor::shared_ptr factor: *graph )
    {
        gtsam::BetweenFactor<gtsam::Pose3>::shared_ptr f = dynamic_pointer_cast<gtsam::BetweenFactor<gtsam::Pose3>>( factor );
        if ( f )
        {
            gtsam::SharedNoiseModel model = f->noiseModel();
            gtsam::noiseModel::Gaussian::shared_ptr gaussianModel = dynamic_pointer_cast<gtsam::noiseModel::Gaussian>( model );
            if ( gaussianModel )
            {
                // write the edge information 
                gtsam::Matrix info = gaussianModel->R().transpose() * gaussianModel->R();
                gtsam::Pose3 pose = f->measured();
                gtsam::Point3 p = pose.translation();
                gtsam::Quaternion q = pose.rotation().toQuaternion();
                fout<<"EDGE_SE3:QUAT "<<f->key1()<<" "<<f->key2()<<" "
                    <<p.x() <<" "<<p.y() <<" "<<p.z() <<" "
                    <<q.x()<<" "<<q.y()<<" "<<q.z()<<" "<<q.w()<<" ";
                gtsam::Matrix infoG2o = gtsam::I_6x6;
                infoG2o.block(0,0,3,3) = info.block(3,3,3,3); // cov translation
                infoG2o.block(3,3,3,3) = info.block(0,0,3,3); // cov rotation
                infoG2o.block(0,3,3,3) = info.block(0,3,3,3); // off diagonal
                infoG2o.block(3,0,3,3) = info.block(3,0,3,3); // off diagonal
                for ( int i=0; i<6; i++ )
                    for ( int j=i; j<6; j++ )
                    {
                        fout<<infoG2o(i,j)<<" ";
                    }
                fout<<endl;
            }
        }
    }
    fout.close();
    cout<<"done."<<endl;
}

In [1]:
import numpy as np
import gtsam
import argparse
import os
import sys

def read_g2o_file(filename):
    """读取 g2o 文件并返回顶点和边的信息"""
    vertices = {}
    edges = []
    
    with open(filename, 'r') as file:
        for line in file:
            data = line.strip().split()
            if not data:
                continue
            
            if data[0] == "VERTEX_SE3:QUAT":
                id = int(data[1])
                pose = gtsam.Pose3(
                    gtsam.Rot3.Quaternion(float(data[7]), float(data[4]), float(data[5]), float(data[6])),
                    gtsam.Point3(float(data[2]), float(data[3]), float(data[4]))
                )
                vertices[id] = pose
            
            elif data[0] == "EDGE_SE3:QUAT":
                id1, id2 = int(data[1]), int(data[2])
                measurement = gtsam.Pose3(
                    gtsam.Rot3.Quaternion(float(data[8]), float(data[5]), float(data[6]), float(data[7])),
                    gtsam.Point3(float(data[3]), float(data[4]), float(data[5]))
                )
                information = np.array([float(x) for x in data[9:]]).reshape(6, 6)
                edges.append((id1, id2, measurement, information))
    
    return vertices, edges

def optimize_pose_graph(vertices, edges):
    """使用 GTSAM 优化位姿图"""
    graph = gtsam.NonlinearFactorGraph()
    initial_estimate = gtsam.Values()

    # 添加先验因子以固定第一个顶点
    first_key = list(vertices.keys())[0]
    prior_noise = gtsam.noiseModel.Diagonal.Sigmas(np.array([1e-6]*6))
    graph.add(gtsam.PriorFactorPose3(first_key, vertices[first_key], prior_noise))

    # 添加顶点和边
    for key, pose in vertices.items():
        initial_estimate.insert(key, pose)

    for edge in edges:
        id1, id2, measurement, information = edge
        noise = gtsam.noiseModel.Gaussian.Information(information)
        graph.add(gtsam.BetweenFactorPose3(id1, id2, measurement, noise))

    # 设置优化参数
    parameters = gtsam.LevenbergMarquardtParams()
    parameters.setVerbosity("ERROR")
    parameters.setMaxIterations(20)
    parameters.setLinearSolverType("MULTIFRONTAL_QR")

    # 执行优化
    optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial_estimate, parameters)
    result = optimizer.optimize()

    print(f"Initial error: {graph.error(initial_estimate)}")
    print(f"Final error: {graph.error(result)}")

    return result

def write_g2o_file(filename, optimized_values, edges):
    """将优化结果写入 g2o 文件"""
    with open(filename, 'w') as file:
        # 写入顶点
        for key in optimized_values.keys():
            pose = optimized_values.atPose3(key)
            q = pose.rotation().quaternion()
            t = pose.translation()
            file.write(f"VERTEX_SE3:QUAT {key} {t.x()} {t.y()} {t.z()} {q[1]} {q[2]} {q[3]} {q[0]}\n")

        # 写入边
        for edge in edges:
            id1, id2, measurement, information = edge
            q = measurement.rotation().quaternion()
            t = measurement.translation()
            file.write(f"EDGE_SE3:QUAT {id1} {id2} {t.x()} {t.y()} {t.z()} {q[1]} {q[2]} {q[3]} {q[0]}")
            for i in range(6):
                for j in range(i, 6):
                    file.write(f" {information[i, j]}")
            file.write("\n")

def main(input_file=None):
    # if input_file is None:
    #     if len(sys.argv) > 1:
    #         input_file = sys.argv[1]
    #     else:
    #         input_file = input("请输入g2o文件路径: ")
    input_file = "/Users/bytedance/Desktop/test/slambook_python/ch11/sphere.g2o"

    if not os.path.exists(input_file):
        print(f"文件 {input_file} 不存在。")
        return

    # 读取输入文件
    vertices, edges = read_g2o_file(input_file)
    print(f"读取了 {len(vertices)} 个顶点和 {len(edges)} 条边。")

    # 优化位姿图
    print("正在优化位姿图...")
    optimized_values = optimize_pose_graph(vertices, edges)

    # 生成输出文件名
    output_file = os.path.splitext(input_file)[0] + "_result.g2o"

    # 写入结果
    write_g2o_file(output_file, optimized_values, edges)
    print(f"优化完成。结果已写入 {output_file}")

if __name__ == "__main__":
    main()



: 

In [ ]:
#  /Users/bytedance/Desktop/test/slambook_python/ch11/sphere.g2o

In [ ]:
# 会导致程序崩溃呀额额额
